# नामित इकाई पहचान (NER)

यह नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) से ली गई है।

इस उदाहरण में, हम सीखेंगे कि [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) Dataset से NER मॉडल को कैसे प्रशिक्षित करें। आगे बढ़ने से पहले, कृपया [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) फ़ाइल को वर्तमान डायरेक्टरी में डाउनलोड करें।


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## डेटासेट तैयार करना

हम डेटासेट को एक डेटा फ्रेम में पढ़ने से शुरू करेंगे। यदि आप Pandas का उपयोग करने के बारे में अधिक जानना चाहते हैं, तो हमारे [Data Science for Beginners](http://aka.ms/datascience-beginners) में [डेटा प्रोसेसिंग पर पाठ](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) देखें।


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

अब हमें शब्दावली के साथ भी यही करना होगा। सरलता के लिए, हम शब्द आवृत्ति को ध्यान में रखे बिना शब्दावली बनाएंगे; वास्तविक जीवन में आप Keras वेक्टराइज़र का उपयोग करना चाह सकते हैं और शब्दों की संख्या को सीमित कर सकते हैं।


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

हमें प्रशिक्षण के लिए वाक्यों का एक डेटासेट बनाना होगा। आइए मूल डेटासेट के माध्यम से लूप करें और सभी व्यक्तिगत वाक्यों को `X` (शब्दों की सूची) और `Y` (टोकन की सूची) में विभाजित करें:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

सादगी के लिए, हम सभी वाक्यों को अधिकतम लंबाई तक 0 टोकन के साथ पैड करेंगे। वास्तविक जीवन में, हम अधिक समझदार रणनीति का उपयोग करना चाह सकते हैं, और केवल एक मिनीबैच के भीतर अनुक्रमों को पैड कर सकते हैं।


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## टोकन वर्गीकरण नेटवर्क परिभाषित करना

हम टोकन वर्गीकरण के लिए दो-लेयर बिडायरेक्शनल LSTM नेटवर्क का उपयोग करेंगे। अंतिम LSTM लेयर के प्रत्येक आउटपुट पर डेंस क्लासिफायर लागू करने के लिए, हम `TimeDistributed` संरचना का उपयोग करेंगे, जो LSTM के प्रत्येक चरण पर उसके प्रत्येक आउटपुट के लिए एक ही डेंस लेयर को दोहराता है:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

यहाँ ध्यान दें कि हम अपने डेटासेट के लिए `maxlen` को स्पष्ट रूप से निर्दिष्ट कर रहे हैं - यदि हम चाहते हैं कि नेटवर्क परिवर्तनीय लंबाई वाले अनुक्रमों को संभाल सके, तो नेटवर्क को परिभाषित करते समय हमें थोड़ा अधिक चतुर होना होगा।

अब चलिए मॉडल को ट्रेन करते हैं। गति के लिए, हम केवल एक एपोक के लिए ट्रेन करेंगे, लेकिन आप इसे अधिक समय तक ट्रेन करने की कोशिश कर सकते हैं। साथ ही, आप डेटासेट के कुछ हिस्से को प्रशिक्षण डेटासेट के रूप में अलग करना चाह सकते हैं, ताकि वैलिडेशन सटीकता का अवलोकन किया जा सके।


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## परिणाम का परीक्षण

अब देखते हैं कि हमारा एंटिटी पहचान मॉडल एक नमूना वाक्य पर कैसे काम करता है:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## मुख्य बातें

यहां तक कि एक साधारण LSTM मॉडल भी NER में उचित परिणाम दिखाता है। हालांकि, बेहतर परिणाम प्राप्त करने के लिए, आप बड़े प्री-ट्रेंड भाषा मॉडल जैसे BERT का उपयोग करना चाह सकते हैं। Huggingface Transformers लाइब्रेरी का उपयोग करके NER के लिए BERT को ट्रेन करने की प्रक्रिया [यहां](https://huggingface.co/course/chapter7/2?fw=pt) वर्णित है।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
